In [1]:
!pip install torch==2.2
!pip install torchvision==0.17
!pip install tensorflow==2.15.0
!pip install onnx2tf==1.19.11
!pip install onnx==1.15.0
!pip install Pillow==9.3.0
!pip install nvidia-pyindex==1.0.9
!pip install onnx-graphsurgeon==0.3.27
!pip install sng4onnx==1.0.1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: pytorch-lightning 1.7.0 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: py

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import numpy as np
from PIL import Image

import tensorflow as tf
import onnx2tf

In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.cn1 = nn.Conv2d(1, 16, 3, 1)
        self.cn2 = nn.Conv2d(16, 32, 3, 1)
        self.dp1 = nn.Dropout2d(0.10)
        self.dp2 = nn.Dropout2d(0.25)
        self.fc1 = nn.Linear(4608, 64) # 4608 is basically 12 X 12 X 32
        self.fc2 = nn.Linear(64, 10)
 
    def forward(self, x):
        x = self.cn1(x)
        x = F.relu(x)
        x = self.cn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dp1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dp2(x)
        x = self.fc2(x)
        op = F.log_softmax(x, dim=1)
        return op
    
model = ConvNet()

In [4]:
PATH_TO_MODEL = "./convnet.pth"
model.load_state_dict(torch.load(PATH_TO_MODEL, map_location="cpu"))

<All keys matched successfully>

In [5]:
image = Image.open("./digit_image.jpg")

In [6]:
def image_to_tensor(image):
    gray_image = transforms.functional.to_grayscale(image)
    resized_image = transforms.functional.resize(gray_image, (28, 28))
    input_image_tensor = transforms.functional.to_tensor(resized_image)
    input_image_tensor_norm = transforms.functional.normalize(input_image_tensor, (0.1302,), (0.3069,))
    return input_image_tensor_norm

In [7]:
input_tensor = image_to_tensor(image)

In [8]:
model.eval()
for p in model.parameters():
    p.requires_grad_(False)

In [9]:
demo_input = torch.ones(1, 1, 28, 28)
torch.onnx.export(model, demo_input, "convnet.onnx")

/Users/Ashish.Jha/anaconda3/envs/python39/lib/python3.9/site-packages/torch/nn/functional.py:1347: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [10]:
# Now, let's convert the ONNX model to TF
onnx2tf.convert(
    input_onnx_file_path="convnet.onnx",
    output_folder_path="convnet_tf",
    non_verbose=True,
)

Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 10, Total Ops 21, % non-converted = 47.62 %
 * 10 ARITH ops

- arith.constant:   10 occurrences  (f32: 8, i32: 2)



  (f32: 2)
  (f32: 2)
  (f32: 1)
  (f32: 1)
  (f32: 1)
  (f32: 1)
Summary on the non-converted ops:
---------------------------------
 * Accepted dialects: tfl, builtin, func
 * Non-Converted Ops: 10, Total Ops 29, % non-converted = 34.48 %
 * 10 ARITH ops

- arith.constant:   10 occurrences  (f16: 8, i32: 2)



  (f32: 2)
  (f32: 8)
  (f32: 2)
  (f32: 1)
  (f32: 1)
  (f32: 1)
  (f32: 1)


In [11]:
# Load the saved model
model = tf.saved_model.load("./convnet_tf/")

model

<ConcreteFunction (inputs_0: TensorSpec(shape=(1, 28, 28, 1), dtype=tf.float32, name='inputs_0')) -> TensorSpec(shape=(1, 10), dtype=tf.float32, name='unknown') at 0x2F71EEBB0>

In [12]:
# Perform inference
output = model(input_tensor.unsqueeze(-1))

# Print the output
print("Model Output:", output)

Model Output: tf.Tensor(
[[-1.04575987e+01 -1.39286604e+01 -2.57339468e-03 -8.81325722e+00
  -1.02665386e+01 -1.58330278e+01 -1.25931921e+01 -1.39404545e+01
  -6.05328608e+00 -1.29597530e+01]], shape=(1, 10), dtype=float32)
